In [1]:
include("Axis_Aligned_Box.jl");
type Mondrian_Node
    parent::Nullable{Mondrian_Node}
    left::Nullable{Mondrian_Node}
    right::Nullable{Mondrian_Node}
    τ::Float64
    node_type::Array{Bool,1}        # node,leaf,root
    δ::Nullable{Int}                # split dimension
    ζ::Nullable{Float64}            # split position
    Θ::Nullable{Axis_Aligned_Box}   # data boxes B
    tab::Array{Int}
    c::Array{Int}
    Gₚ::Array{Float64}
end

function Mondrian_Node()
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      0,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64)
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64, node_type::Array{Bool,1})
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,
                      node_type,
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

type Mondrian_Tree
    root::Nullable{Mondrian_Node}
    leaves::Array{Mondrian_Node,1}
end

function Mondrian_Tree()
    T = Mondrian_Tree(Nullable{Mondrian_Node}(),Array{Mondrian_Node,1}())
end

function Mondrian_Tree(N::Mondrian_Node)
    T = Mondrian_Tree(N,Array{Mondrian_Node,1}())
end

Mondrian_Tree

In [42]:
using Distributions

function Sample_Mondrian_Tree(λ,D)
    δ = []
    ζ = []
    n = 1:size(D,1)
    e = Mondrian_Node(0.0,[false,false,true])
    T = Mondrian_Tree(e)
    Θ = Axis_Aligned_Box(get_intervals(D[1]))
    e.Θ = Θ
    e.tab = zeros(size(unique(D[2]),1))
    e.c = zeros(size(unique(D[2]),1))
    e.Gₚ = zeros(size(unique(D[2]),1))
    Sample_Mondrian_Block(e, Θ, λ, δ, ζ, T)
    return T
end

function Sample_Mondrian_Block(j, Θ, λ, δ, ζ, tree)
    E = rand(Exponential(1/Linear_dimension(Θ)))
    if j.node_type[3]==true
        τₚ = 0
    else
        τₚ = (get(j.parent)).τ
    end
    if τₚ + E < λ
        d,x = sample_split_dimension(Θ)
        j.δ = d
        j.ζ = x
        Θᴸ = copy(Θ)
        Θᴿ = copy(Θ)
        Θᴸ.Intervals[d,2]=x
        Θᴿ.Intervals[d,1]=x
        left = Mondrian_Node(τₚ+E, [true,false,false])
        left.parent = j
        left.Θ = Θᴸ
        left.tab = zeros(size(j.tab))
        left.c = zeros(size(j.tab))
        left.Gₚ = zeros(size(unique(D[2]),1))
        right = Mondrian_Node(τₚ+E, [true,false,false])
        right.parent = j
        right.Θ = Θᴿ
        right.tab = zeros(size(j.tab))
        right.c = zeros(size(j.tab))
        right.Gₚ=zeros(size(unique(D[2]),1))
        j.left = left
        j.right = right
        Sample_Mondrian_Block(left, Θᴸ, λ, δ, ζ, tree)
        Sample_Mondrian_Block(right, Θᴿ,λ, δ, ζ, tree)
    else
        j.τ = λ
        j.node_type = [false,true,false]
        push!(tree.leaves,j)
    end
end

function get_data_indices(Θ::Axis_Aligned_Box, D::Array{Float64,2})
    indices = []
    include = false
    for i in 1:size(D,1)
        for j in 1:size(Θ.Intervals,1)
            if (D[i,j] < Θ.Intervals[j,1] || D[i,j] > Θ.Intervals[j,2])
                include = false
                break
            end
            include = true
        end
        if (include)    
            push!(indices, i)
        end
    end
    return indices
end

function update_posterior_counts(leaf,y)
    leaf.c[y] += 1
    l = leaf
    while true
        if l.tab[y] == 1
            return
        else
            if l.node_type[2] == false
                l.c[y] = get(l.left).tab[y]+get(l.right).tab[y]
            l.tab[y] = min(l.c[y],1)
            end
            if l.node_type[3] == true
                return
            else
                l = get(l.parent)
            end
        end
    end         
end

function update_posterior_counts_batch(Tree,D)
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),D[1])
        for y in D[2][indices,:]
            update_posterior_counts(leaf,y)
        end
    end
end

function compute_predictive_posterior_distribution(Tree, γ)
    J = [get(Tree.root)]
    while (size(J,1) != 0)
        j = shift!(J)
        if (j.node_type[3]==true)
            p = ones(length(j.c))./length(j.c)
            d = exp(-γ*(j.τ))
        else 
            d = exp(-γ*(j.τ-get(j.parent).τ))
            p = get(j.parent).Gₚ
        end
        for k in 1:length(j.c)
            println(sum(j.c))
            j.Gₚ[k] = (1/(sum(j.c)+1))*(j.c[k]-d*j.tab[k]+sum(j.tab)*p[k])
        end
        if j.node_type[2] == false 
            push!(J, get(j.left))
            push!(J, get(j.right))
        end
    end
end

function expected_discount(γ, nⱼ, Δⱼ)
    Δ = rand(Truncated(Exponential(nⱼ),0,Δⱼ),1000)
    return mean(exp.(-γ*Δ))
end

function predict(T,x)
    j = get(T.root)
    not_sep = 1
    s = zeros(size(j.c,1))
    while true
        if (j.node_type[3] == true)
            Δⱼ = j.τ
        else
            Δⱼ = j.τ - get(j.parent).τ
        end
        nⱼ=0
        for d in size(x,1)
            nⱼ += max(x[d]-get(j.Θ).Intervals[d,2],0) + max(get(j.Θ).Intervals[d,1]-x[d],0)
        end
        pⱼ = 1-exp(Δⱼ*nⱼ)
        if pⱼ > 0
            d = expected_discount(γ, nⱼ, Δⱼ)
        end
    end
end

compute_predictive_posterior_distribution (generic function with 1 method)

In [43]:
X = rand(100,4)
Y = rand([1,2],size(X,1))
D = (X,Y)

([0.624896 0.494893 0.534358 0.339575; 0.0983008 0.86944 0.56858 0.239881; … ; 0.995278 0.848457 0.311969 0.211594; 0.868295 0.293071 0.537087 0.705959], [2, 2, 1, 1, 2, 2, 1, 1, 2, 2  …  2, 2, 1, 2, 2, 2, 2, 1, 2, 1])

In [44]:
MT = Sample_Mondrian_Tree(0.3,D);
update_posterior_counts_batch(MT,D)

In [45]:
compute_predictive_posterior_distribution(MT,1);

0
0
0
0
0
0
8
8
0
0
0
0
0
0
0
0
0
0
58
58
0
0
5
5
6
6
5
5
17
17
0
0
1
1


In [287]:
a.Intervals

LoadError: [91mtype Mondrian_Node has no field Intervals[39m

In [43]:
D[1][1,2] < a.Intervals[2,1] || D[1][1,2] > a.Intervals[2,2]

false

In [40]:
D[1,2]

LoadError: [91mMethodError: no method matching getindex(::Tuple{Array{Float64,2},Array{Int64,1}}, ::Int64, ::Int64)[39m

In [22]:
Y = [1,2,2,2,1]

5-element Array{Int64,1}:
 1
 2
 2
 2
 1

In [24]:
size(Y[Y.==1],1)

2

In [16]:
ones(2)

2-element Array{Float64,1}:
 1.0
 1.0

In [17]:
exp(2)

7.38905609893065

In [19]:
sum([1,2,3])

6